In [143]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [144]:
import jinja2
from anthropic import AnthropicBedrock
from core import stages
import json


In [145]:
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [146]:
environment = jinja2.Environment()

In [147]:
tsp_tpl = environment.from_string(stages.typespec.PROMPT)
dzl_tpl = environment.from_string(stages.drizzle.PROMPT)
rtr_tpl = environment.from_string(stages.router.PROMPT)
hdl_tpl = environment.from_string(stages.handlers.PROMPT)
pre_tpl = environment.from_string(stages.processors.PROMPT_PRE)

In [148]:
application_description = """
Bot that tracks my exercise routine in the gym, tracks progress and suggests new routines
for specific list of available equipment and time constraints.
""".strip()

In [173]:
from shutil import copytree, ignore_patterns


copytree('./templates/', './app_output', ignore=ignore_patterns('*.pyc', '__pycache__')) # 'node_modules'


'./app_output'

In [149]:
prompt = tsp_tpl.render(
    application_description=application_description,
)

tsp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
typespec = stages.typespec.parse_output(tsp_response.content[0].text)

In [150]:
typespec

{'reasoning': 'For a gym tracking bot, I expect users to send messages like:\n"I did 3 sets of bench press with 80kg today" or \n"Plan me a 45-minute workout for chest using dumbbells and bench"\n\nKey functions needed:\n1. Record workout sessions with exercises, sets, reps, and weights\n2. Plan new routines based on available equipment and time\n3. Track progress over time\n\nThe LLM can extract:\n- Exercise details from free-form text\n- Available equipment from user descriptions\n- Time constraints for workout planning\n- Progress tracking periods\n\nEquipment and exercises need to be predefined to ensure consistency in tracking.',
 'typespec_definitions': 'model Exercise {\n  name: string;\n  sets: integer;\n  reps: integer;\n  weight: float;\n  notes: string;\n}\n\nmodel WorkoutSession {\n  date: utcDateTime;\n  duration: duration;\n  exercises: Exercise[];\n}\n\nmodel Equipment {\n  name: string;\n  category: string;\n}\n\nmodel WorkoutPlan {\n  targetDuration: duration;\n  exerc

In [151]:
print(typespec["reasoning"])

For a gym tracking bot, I expect users to send messages like:
"I did 3 sets of bench press with 80kg today" or 
"Plan me a 45-minute workout for chest using dumbbells and bench"

Key functions needed:
1. Record workout sessions with exercises, sets, reps, and weights
2. Plan new routines based on available equipment and time
3. Track progress over time

The LLM can extract:
- Exercise details from free-form text
- Available equipment from user descriptions
- Time constraints for workout planning
- Progress tracking periods

Equipment and exercises need to be predefined to ensure consistency in tracking.


In [152]:
print(typespec["typespec_definitions"])

model Exercise {
  name: string;
  sets: integer;
  reps: integer;
  weight: float;
  notes: string;
}

model WorkoutSession {
  date: utcDateTime;
  duration: duration;
  exercises: Exercise[];
}

model Equipment {
  name: string;
  category: string;
}

model WorkoutPlan {
  targetDuration: duration;
  exercises: Exercise[];
  equipment: Equipment[];
}

interface GymBot {
  @llm_func(2)
  recordWorkout(workout: WorkoutSession): void;

  @llm_func(3)
  planWorkout(duration: duration, equipment: Equipment[]): WorkoutPlan;

  @llm_func(1)
  trackProgress(exerciseName: string, from: utcDateTime, to: utcDateTime): Exercise[];
}


In [153]:
prompt = dzl_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
)

dzl_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
drizzle = stages.drizzle.parse_output(dzl_response.content[0].text)

In [154]:
print(drizzle["drizzle_schema"])

import { serial, text, integer, real, timestamp, pgTable, primaryKey } from "drizzle-orm/pg-core";

export const exercisesTable = pgTable("exercises", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  sets: integer("sets").notNull(),
  reps: integer("reps").notNull(),
  weight: real("weight").notNull(),
  notes: text("notes"),
});

export const equipmentTable = pgTable("equipment", {
  id: serial("id").primaryKey(),
  name: text("name").notNull(),
  category: text("category").notNull(),
});

export const workoutSessionsTable = pgTable("workout_sessions", {
  id: serial("id").primaryKey(),
  date: timestamp("date").notNull(),
  duration_minutes: integer("duration_minutes").notNull(),
});

export const workoutPlansTable = pgTable("workout_plans", {
  id: serial("id").primaryKey(),
  target_duration_minutes: integer("target_duration_minutes").notNull(),
});

// Junction table for workout_sessions and exercises
export const sessionExercisesTable = pgTable("session_exerci

In [155]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = stages.router.TOOLS
)

funcs = stages.router.parse_outputs([content for content in exp_response.content])["user_functions"]


In [156]:
funcs

[{'name': 'recordWorkout',
  'description': 'Records a complete workout session including exercises performed, their details (sets, reps, weight), duration, and date. Use when user wants to log or save their workout/exercise activities.',
  'examples': ['I want to record my exercise routine for today',
   'Let me log my workout from this morning',
   'I did 3 sets of bench press with 10 reps each at 150 pounds',
   'Save my gym session: 30 minutes cardio and weight training']},
 {'name': 'planWorkout',
  'description': 'Creates a workout plan based on specified duration and available equipment. Use when user wants suggestions or a plan for their workout.',
  'examples': ['Can you create a 45-minute workout plan for me?',
   'Plan a workout routine using dumbbells and a bench',
   'I need a workout plan for my home gym equipment',
   'Make me an exercise plan for 1 hour using basic equipment']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specif

In [157]:
pre_processors = {}
for function_name in typespec["llm_functions"]:
    prompt = pre_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
    )

    pre_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    pre_processor = stages.processors.parse_output(pre_response.content[0].text)
    pre_processors[function_name] = pre_processor

In [158]:
print(pre_response.content[0].text)

<instructions>
The trackProgress function requires three arguments:
1. exerciseName: A string representing the exercise to track (required)
2. from: A UTC datetime marking the start of tracking period
3. to: A UTC datetime marking the end of tracking period

If dates are not specified, default to:
- 'to' = current datetime
- 'from' = 30 days before 'to'

Convert natural language exercise names to standard names (e.g., "bench" → "bench press")
</instructions>

<examples>
    <example>
        <input>Show my bench press progress</input>
        <output>{
            "exerciseName": "bench press",
            "from": "2024-01-15T00:00:00Z",
            "to": "2024-02-14T00:00:00Z"
        }</output>
    </example>

    <example>
        <input>Track my squats from January 1st</input>
        <output>{
            "exerciseName": "squats",
            "from": "2024-01-01T00:00:00Z",
            "to": "2024-02-14T00:00:00Z"
        }</output>
    </example>

    <example>
        <input>How

In [159]:
pre_processors

{'recordWorkout': {'instructions': 'For recordWorkout function:\n- Date defaults to current timestamp if not specified\n- Duration should be parsed from explicit mentions or estimated based on exercise count (approximately 3-5 minutes per exercise)\n- Exercise entries require at minimum: name and either sets/reps or total count\n- Weight is optional and can be in kg or lbs (default to kg if not specified)\n- Notes field can capture additional context or form details\n- Multiple exercises can be included in a single workout session',
  'examples': [('Just finished bench press, 3 sets of 8 reps at 80kg, felt strong today',
    '{\n  "workout": {\n    "date": "2024-01-20T14:30:00Z",\n    "duration": "PT15M",\n    "exercises": [{\n      "name": "bench press",\n      "sets": 3,\n      "reps": 8,\n      "weight": 80.0,\n      "notes": "felt strong today"\n    }]\n  }\n}'),
   ('Completed full body workout: squats 5x5 100kg, deadlifts 3x5 120kg, pulled my back a bit',
    '{\n  "workout": {\n

In [160]:
handlers = {}
for function_name in typespec["llm_functions"]:
    prompt = hdl_tpl.render(
        function_name=function_name,
        typespec_definitions=typespec["typespec_definitions"],
        drizzle_schema=drizzle["drizzle_schema"],
    )

    hdl_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": prompt}]
    )
    handlers[function_name] = stages.handlers.parse_output(hdl_response.content[0].text)


In [184]:
handlers

{'recordWorkout': {'handler': 'import { db } from "../db";\nimport { \n    workoutSessionsTable, \n    exercisesTable, \n    sessionExercisesTable \n} from \'../db/schema/application\';\n\nconst handle = async (workout: WorkoutSession): Promise<void> => {\n    // Insert the workout session\n    const [workoutSession] = await db\n        .insert(workoutSessionsTable)\n        .values({\n            date: workout.date,\n            duration_minutes: Math.floor(workout.duration / 60), // Convert duration to minutes\n        })\n        .returning({ id: workoutSessionsTable.id });\n\n    // Insert exercises and create session-exercise relationships\n    for (const exercise of workout.exercises) {\n        // Insert the exercise\n        const [newExercise] = await db\n            .insert(exercisesTable)\n            .values({\n                name: exercise.name,\n                sets: exercise.sets,\n                reps: exercise.reps,\n                weight: exercise.weight,\n         

In [164]:
with open("./templates/interpolation/handler.tpl", "r") as f:
    handler_ts_tpl = environment.from_string(f.read())

In [166]:
hdl_key = "recordWorkout"
params = {
    "handler": handlers[hdl_key]["handler"],
    "instructions": pre_processors[hdl_key]["instructions"],
    "examples": pre_processors[hdl_key]["examples"],
}
file_content = handler_ts_tpl.render(**params)

In [167]:
print(file_content)

import { Message } from "../common/handler";
import { client } from "../common/llm";
import { db } from "../db";
import { 
    workoutSessionsTable, 
    exercisesTable, 
    sessionExercisesTable 
} from '../db/schema/application';

const handle = async (workout: WorkoutSession): Promise<void> => {
    // Insert the workout session
    const [workoutSession] = await db
        .insert(workoutSessionsTable)
        .values({
            date: workout.date,
            duration_minutes: Math.floor(workout.duration / 60), // Convert duration to minutes
        })
        .returning({ id: workoutSessionsTable.id });

    // Insert exercises and create session-exercise relationships
    for (const exercise of workout.exercises) {
        // Insert the exercise
        const [newExercise] = await db
            .insert(exercisesTable)
            .values({
                name: exercise.name,
                sets: exercise.sets,
                reps: exercise.reps,
                weight: e

In [170]:
print(handlers["planWorkout"]["handler"])

import { db } from "../db";
import { 
    workoutPlansTable, 
    planExercisesTable, 
    planEquipmentTable,
    exercisesTable,
    equipmentTable 
} from '../db/schema/application';
import { eq } from 'drizzle-orm';

const handle = async (duration: number, equipment: { name: string, category: string }[]): Promise<{
    targetDuration: number,
    exercises: Array<{
        name: string,
        sets: number,
        reps: number,
        weight: number,
        notes: string
    }>,
    equipment: Array<{
        name: string,
        category: string
    }>
}> => {
    // Create the workout plan
    const [workoutPlan] = await db
        .insert(workoutPlansTable)
        .values({
            target_duration_minutes: duration
        })
        .returning();

    // Insert or get equipment IDs
    const equipmentIds = await Promise.all(
        equipment.map(async (eq) => {
            const [existingEquipment] = await db
                .select()
                .from(equipmentT

In [191]:
import os

with open("./templates/interpolation/handler.debug.tpl", "r") as f:
    handler_debug_tpl = environment.from_string(f.read())
    
    for handler_name in handlers.keys():
        params = {
            "handler": {"name": handler_name},
            #"instructions": pre_processors[hdl_key]["instructions"],
            #"examples": pre_processors[hdl_key]["examples"],
        }
        file_content = handler_debug_tpl.render(**params)

        # Convert PascalCase to snake_case for file naming
        handler_snake_name = ''.join(['_' + c.lower() if c.isupper() else c for c in handler_name]).lstrip('_')
        handler_file_name = f'{handler_snake_name}_handler_debug'
        
        with open(os.path.join('./app_output/app_schema/src/handlers', handler_file_name + '.ts'), 'w') as f:
            f.write(file_content)
        
        print(f"Handler: {handler_file_name}")
        print(file_content)


Handler: record_workout_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler recordWorkout executed';
    return [{ role: 'assistant', content: content }];
};

export const recordWorkout = new GenericHandler<[string], string>(handle, preProcessor, postProcessor);
Handler: plan_workout_handler_debug
import { GenericHandler, Message } from "../common/handler";
import { client } from "../common/llm";

const preProcessor = async (input: Message[]): Promise<[string]> => {
    return [''];
};

const handle = (input: string): string => {
    return '';
};

const postProcessor = (output: string): Message[] => {
    const content = 'handler planWorkout executed';
    return [{ role: 'assistant', con

In [189]:
handlers.keys()

dict_keys(['recordWorkout', 'planWorkout', 'trackProgress'])

In [186]:
handlers

{'recordWorkout': {'handler': 'import { db } from "../db";\nimport { \n    workoutSessionsTable, \n    exercisesTable, \n    sessionExercisesTable \n} from \'../db/schema/application\';\n\nconst handle = async (workout: WorkoutSession): Promise<void> => {\n    // Insert the workout session\n    const [workoutSession] = await db\n        .insert(workoutSessionsTable)\n        .values({\n            date: workout.date,\n            duration_minutes: Math.floor(workout.duration / 60), // Convert duration to minutes\n        })\n        .returning({ id: workoutSessionsTable.id });\n\n    // Insert exercises and create session-exercise relationships\n    for (const exercise of workout.exercises) {\n        // Insert the exercise\n        const [newExercise] = await db\n            .insert(exercisesTable)\n            .values({\n                name: exercise.name,\n                sets: exercise.sets,\n                reps: exercise.reps,\n                weight: exercise.weight,\n         

In [178]:
with open("./templates/interpolation/logic_index.tpl", "r") as f:
    logic_index_tpl = environment.from_string(f.read())
    
    params = {
        "handlers": handlers
    }
    file_content = logic_index_tpl.render(**params)
    
    with open(os.path.join('./app_output/app_schema/src/logic/index.ts'), 'w') as f:
        f.write(file_content)
            
    print(file_content)

import { GenericHandler } from "../common/handler";

import { recordWorkout } from "../handlers/record_workout_handler_debug";

import { planWorkout } from "../handlers/plan_workout_handler_debug";

import { trackProgress } from "../handlers/track_progress_handler_debug";


export const handlers: {[key: string]: GenericHandler<any[], any>} = {
    
    'recordWorkout': recordWorkout,
    
    'planWorkout': planWorkout,
    
    'trackProgress': trackProgress,
    
};


In [24]:
ts_defs = stages.typespec.parse_output(stages.typespec.PROMPT)

In [25]:
ts_defs

{'reasoning': 'I expect user to send messages like "I ate a burger" or "I had a salad for lunch".\nLLM can extract and infer the arguments from plain text and pass them to the handler\n"I ate a burger" -> recordDish({name: "burger", ingredients: [\n    {name: "bun", calories: 200},\n    {name: "patty", calories: 300},\n    {name: "lettuce", calories: 10},\n    {name: "tomato", calories: 20},\n    {name: "cheese", calories: 50},\n]})\n- recordDish(dish: Dish): void;\n...',
 'typespec_definitions': 'model Dish {\n    name: string;\n    ingredients: Ingredient[];\n}\n\nmodel Ingredient {\n    name: string;\n    calories: integer;\n}\n\ninterface DietBot {\n    @llm_func(1)\n    recordDish(dish: Dish): void;\n    @llm_func(1)\n    listDishes(from: utcDateTime, to: utcDateTime): Dish[];\n}',
 'llm_functions': ['recordDish', 'listDishes']}

In [26]:
func_names = stages.typespec.extract_llm_func_names(stages.typespec.PROMPT)

In [27]:
func_names

['recordDish', 'listDishes']

# Project Generation

In [28]:
from shutil import copytree, ignore_patterns

In [29]:
def setup_project(workdir: str):
    copytree('templates', workdir, ignore=ignore_patterns('*.pyc', '__pycache__', 'node_modules'))

In [30]:
setup_project('test')

FileExistsError: [Errno 17] File exists: 'test'

In [31]:
from core.stages import router

In [32]:
prompt = rtr_tpl.render(
    typespec_definitions=typespec["typespec_definitions"],
    user_request="I want to record my exercise routine for today."
)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}],
    tools = router.TOOLS
)

funcs = router.parse_outputs([content for content in exp_response.content])["user_functions"]

In [33]:
funcs

[{'name': 'recordExercise',
  'description': 'Records a single exercise performed by the user with details like sets, reps, weight, equipment used, and targeted muscle groups. Use this when the user wants to log or record their exercise activity.',
  'examples': ['I want to record my exercise routine for today',
   'I just did 3 sets of bench press',
   'Log my workout: 4 sets of squats with 200lbs',
   'Add my deadlift session to my log']},
 {'name': 'trackProgress',
  'description': 'Retrieves historical exercise data for a specific exercise over a given time period. Use this when the user wants to see their progress or history for a particular exercise.',
  'examples': ['Show me my bench press progress over the last month',
   'How has my squat improved since January?',
   'What were my deadlift numbers from last week?',
   'Track my progress on pull-ups']},
 {'name': 'suggestWorkout',
  'description': 'Generates a workout plan based on desired duration, target muscle groups, and av

In [47]:
from core.stages import typespec
from core.stages import tsp_compiler

tsp_tpl = environment.from_string(stages.typespec.PROMPT)

typespec_errors = ""
attempts_left = 3

while attempts_left > 0:
    attempts_left -= 1
    
    prompt = tsp_tpl.render(
        application_description=application_description,
        typespec_errors=typespec_errors,
    )
  
    if attempts_left >= 2:
        updated_prompt = prompt + "// PS. Make sure to eject a random error in TypeSpec output for testing purposes"
    else:
        updated_prompt = prompt
  
    tsp_response = client.messages.create(
        model="anthropic.claude-3-5-sonnet-20241022-v2:0",
        max_tokens=8192,
        messages=[{"role": "user", "content": updated_prompt}]
    )
    
    typespec = stages.typespec.parse_output(tsp_response.content[0].text)

    # Write TypeSpec definitions to file
    with open('./test/tsp_schema/notebook.tsp', 'w') as f:
        f.writelines(['import "./helpers.js";', 
                  '\n', 
                  'extern dec llm_func(target: unknown, history: valueof int32);'
                  ])
        f.write(typespec["typespec_definitions"])

    compiler = tsp_compiler.TypeSpecCompiler('./test/tsp_schema/')
    result = compiler.compile('./notebook.tsp')
    
    print(result)
    
    if result["result"] == tsp_compiler.TypeSpecCompilationStatus.COMPILATION_ERROR:
        typespec_errors = result["errors"]
        print(typespec_errors)
        continue
    else:
        break


In [90]:
typespec


{'reasoning': 'For a finance tracking app, users would likely send messages like:\n- "I spent $50 on groceries yesterday"\n- "Got my salary of $3000 today"\n- "Show me my expenses for last month"\n\nThe LLM would need to extract:\n1. Transaction type (expense/income)\n2. Amount\n3. Category\n4. Date\n5. Optional description/notes\n\nFor reports, users might ask for:\n- Time-based summaries\n- Category-based breakdowns\n- Balance calculations\n\nThe interface should support recording transactions and generating reports while keeping the argument structure simple enough to be extracted from natural language.',
 'typespec_definitions': 'model Transaction {\n  type: string; // "expense" or "income"\n  amount: decimal;\n  category: string;\n  date: utcDateTime;\n  description: string;\n}\n\nmodel TransactionSummary {\n  totalIncome: decimal;\n  totalExpenses: decimal;\n  balance: decimal;\n  transactions: Transaction[];\n}\n\ninterface FinanceTracker {\n  @llm_func(1)\n  recordTransaction(t

In [ ]:
prompt = exp_tpl.render(application_description=application_description)

exp_response = client.messages.create(
    model="anthropic.claude-3-5-sonnet-20241022-v2:0",
    max_tokens=8192,
    messages=[{"role": "user", "content": prompt}]
)
expansion = stages.expansion.parse_output(exp_response.content[0].text)

In [ ]:
print(expansion["application_specification"])

<types>
        <type>exercise
            - name
            - equipment_required
            - sets
            - reps
            - weight</type>
        <type>equipment
            - name
            - type</type>
        <type>workout_session
            - date
            - duration
            - exercises_performed</type>
        <type>routine
            - exercises
            - duration
            - equipment_needed</type>
    </types>
    <operations>
        <operation>record_workout_session
            - date
            - exercises with sets/reps/weights</operation>
        <operation>track_progress
            - exercise
            - time period</operation>
        <operation>list_available_equipment</operation>
        <operation>generate_routine
            - available equipment
            - time constraint
            - fitness level</operation>
        <operation>view_history
            - time period</operation>
    </operations>
